<a href="https://colab.research.google.com/github/nagabathula/C255-DataMIning/blob/main/assignment1/Assignment1_kmeans_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kmeans-pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from kmeans_pytorch import kmeans, kmeans_predict
import pandas as pd

In [5]:
df = pd.read_csv('/content/Live.csv')

In [6]:
# drop redundant columns

df.drop(['Column1', 'Column2', 'Column3', 'Column4'], axis=1, inplace=True)

# drop status_id, status_published

df.drop(['status_id', 'status_published'], axis=1, inplace=True)

In [21]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7050 entries, 0 to 7049
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   status_type    7050 non-null   int64
 1   num_reactions  7050 non-null   int64
 2   num_comments   7050 non-null   int64
 3   num_shares     7050 non-null   int64
 4   num_likes      7050 non-null   int64
 5   num_loves      7050 non-null   int64
 6   num_wows       7050 non-null   int64
 7   num_hahas      7050 non-null   int64
 8   num_sads       7050 non-null   int64
 9   num_angrys     7050 non-null   int64
dtypes: int64(10)
memory usage: 550.9 KB


In [14]:
#Convert categorical variable into integers

X = df
y = df['status_type']

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X['status_type'] = le.fit_transform(X['status_type'])
y = le.transform(y)

In [15]:
#Feature Scaling

cols = X.columns
from sklearn.preprocessing import MinMaxScaler
ms = MinMaxScaler()

X = ms.fit_transform(X)
X = pd.DataFrame(X, columns=[cols])
X.head()

,status_type,num_reactions,num_comments,num_shares,num_likes,num_loves,num_wows,num_hahas,num_sads,num_angrys
0,1.000000,0.112314,0.024393,0.076519,0.091720,0.140030,0.010791,0.006369,0.019608,0.0
1,0.333333,0.031847,0.000000,0.000000,0.031847,0.000000,0.000000,0.000000,0.000000,0.0
2,1.000000,0.048195,0.011243,0.016647,0.043312,0.031963,0.003597,0.006369,0.000000,0.0
3,0.333333,0.023567,0.000000,0.000000,0.023567,0.000000,0.000000,0.000000,0.000000,0.0
4,0.333333,0.045223,0.000000,0.000000,0.043312,0.013699,0.000000,0.000000,0.000000,0.0


In [10]:
# set device
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')


In [27]:
df_tensor = torch.tensor(X.to_numpy())

In [30]:
from torch.utils.data import random_split 

train_data, test_data = random_split(df_tensor , [6050, 1000])

print(train_data.dataset)

tensor([[1.0000e+00, 1.1231e-01, 2.4393e-02,  ..., 6.3694e-03, 1.9608e-02,
         0.0000e+00],
        [3.3333e-01, 3.1847e-02, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [1.0000e+00, 4.8195e-02, 1.1243e-02,  ..., 6.3694e-03, 0.0000e+00,
         0.0000e+00],
        ...,
        [3.3333e-01, 4.2463e-04, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [3.3333e-01, 7.4522e-02, 5.7170e-04,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [3.3333e-01, 3.6093e-03, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]], dtype=torch.float64)


In [31]:
from typing import Any
cluster_ids_x, cluster_centers = kmeans(X=train_data.dataset, num_clusters=4, distance='euclidean', device=device)

running k-means on cuda:0..


[running kmeans]: 5it [00:00, 247.41it/s, center_shift=0.000053, iteration=5, tol=0.000100]


In [32]:
print(cluster_ids_x)
print(cluster_centers)

tensor([2, 3, 2,  ..., 3, 3, 3])
tensor([[5.6984e-01, 2.2789e-01, 2.1139e-03, 2.5817e-03, 2.2684e-01, 4.1410e-03,
         6.3207e-03, 1.3750e-03, 4.7619e-03, 5.1203e-04],
        [0.0000e+00, 7.8587e-02, 2.7148e-04, 1.2841e-03, 7.8475e-02, 4.5904e-04,
         6.8517e-04, 2.0220e-04, 0.0000e+00, 0.0000e+00],
        [1.0000e+00, 5.2130e-02, 3.1116e-02, 3.4283e-02, 4.3401e-02, 5.5062e-02,
         8.9638e-03, 1.1250e-02, 8.2604e-03, 8.3424e-03],
        [3.3333e-01, 1.8795e-02, 6.5407e-04, 5.1376e-04, 1.8334e-02, 1.8925e-03,
         1.9977e-03, 1.1316e-03, 2.8900e-03, 1.5478e-03]])


In [33]:
cluster_ids_y = kmeans_predict(
    test_data.dataset, cluster_centers, 'euclidean', device=device
)

predicting on cuda:0..


In [34]:
print(cluster_ids_y)

tensor([2, 3, 2,  ..., 3, 3, 3])
